In [811]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [812]:
import math
import numpy as np
import json
from copy import deepcopy
import bisect

from config import filename
print(filename)

data/custom.json


In [813]:

def n2letter(n):
    '''0 to 'a', 1 to 'b', ... '''
    return chr(96+n)

def string2duration(string):
    ''' "01:50:19.3177493" to duration in seconds'''
    return 3600*int(string[:2]) + 60*int(string[3:5]) + int(string[6:8])   #Duration is int

In [814]:
def read_data(path):
    global task_count
    global tasks
    file = open(path)
    data = json.load(file)
    nodes = data['nodes']
    tasks = dict()
    for task_str, info in nodes.items():
        task = int(task_str)
        tasks[task] = {'Data' : string2duration(info['Data']), 'Dependencies' : info['Dependencies']}
    task_count = len(tasks)
    print("Data loaded successfully. Number of tasks: " + str(task_count))

n_cores = 2
read_data(filename)
tasks

Data loaded successfully. Number of tasks: 20


{1: {'Data': 6734, 'Dependencies': []},
 2: {'Data': 3249, 'Dependencies': [1]},
 3: {'Data': 1171, 'Dependencies': [2]},
 4: {'Data': 4239, 'Dependencies': [2]},
 5: {'Data': 5310, 'Dependencies': [2]},
 6: {'Data': 7143, 'Dependencies': [3]},
 7: {'Data': 5219, 'Dependencies': [3]},
 8: {'Data': 3585, 'Dependencies': [4]},
 9: {'Data': 3860, 'Dependencies': [9]},
 10: {'Data': 6140, 'Dependencies': [7, 8]},
 11: {'Data': 825, 'Dependencies': [10]},
 12: {'Data': 6882, 'Dependencies': [9]},
 13: {'Data': 7698, 'Dependencies': [12]},
 14: {'Data': 5715, 'Dependencies': [1]},
 15: {'Data': 2370, 'Dependencies': [14]},
 16: {'Data': 746, 'Dependencies': [13, 15]},
 17: {'Data': 4021, 'Dependencies': [11]},
 18: {'Data': 3366, 'Dependencies': [15]},
 19: {'Data': 2731, 'Dependencies': [18]},
 20: {'Data': 7847, 'Dependencies': [17, 19]}}

In [815]:
#Tasks to child tasks / Tasks to parents / Task is terminal / Task is inital
task2childs = {task : list() for task in tasks}
task2parents = {task : list() for task in tasks}
for task, info in tasks.items():
    #Add childs
    list_task_parents = info['Dependencies']
    for task_parent in list_task_parents:
        task2childs[task_parent].append(task)
    #Add parents
    task2parents[task] = tasks[task]['Dependencies']
    
def task_is_terminal(task: int):
    return len(task2childs[task]) == 0
def task_is_inital(task: int):
    return len(task2parents[task]) == 0

print(task2childs)
print(task2parents)

{1: [2, 14], 2: [3, 4, 5], 3: [6, 7], 4: [8], 5: [], 6: [], 7: [10], 8: [10], 9: [9, 12], 10: [11], 11: [17], 12: [13], 13: [16], 14: [15], 15: [16, 18], 16: [], 17: [20], 18: [19], 19: [20], 20: []}
{1: [], 2: [1], 3: [2], 4: [2], 5: [2], 6: [3], 7: [3], 8: [4], 9: [9], 10: [7, 8], 11: [10], 12: [9], 13: [12], 14: [1], 15: [14], 16: [13, 15], 17: [11], 18: [15], 19: [18], 20: [17, 19]}


In [816]:
task2sbl = {}

def save_static_bottom_level(task : int):
    task_duration = tasks[task]["Data"]
    if task_is_terminal(task):
        sbl = task_duration
    else:
        list_sbl_child = list()
        for task_child in task2childs[task]:
            if task_child in task2sbl:
                sbl_child = task2sbl[task_child]
            else:
                sbl_child = save_static_bottom_level(task_child)
            list_sbl_child.append(sbl_child)
        sbl = max(list_sbl_child) + task_duration
                
    task2sbl[task] = sbl
    return sbl

for task in tasks:
    if task_is_inital(task):
        save_static_bottom_level(task)
        
task2sbl



{6: 7143,
 20: 7847,
 17: 11868,
 11: 12693,
 10: 18833,
 7: 24052,
 3: 25223,
 8: 22418,
 4: 26657,
 5: 5310,
 2: 29906,
 16: 746,
 19: 10578,
 18: 13944,
 15: 16314,
 14: 22029,
 1: 36640}

In [817]:
class Graph:
    
    def __init__(self):
        self.tasks = tasks
        self.tasks_to_sbl = task2sbl
        self.tasks_to_parent = task2parents
        self.tasks_to_child = task2childs
        self.n_cores = 2
        self.nodes = list()

graph = Graph()

In [818]:
class Node():
    graph = graph
    
    def __init__(self, parent = None, task_to_add = None, core_where_to_add = None, time_task_start = None):
        '''Create a Node object ie a partial scheduling
        parent = parent Node, None if root
        task_to_add : task added to the partial schedule
        core_where_to_add : core where to do task
        time_task_start : instant where the core will start computing the task
        '''        
        if parent is None:
            self.parent = None
            self.tasks_done_time = dict()
            self.cores = {core_n : {"task" : -1, "task_end_time" : 0} for core_n in range(n_cores)}
            
            self.g = 0
            self.f = self.h()
                   
            self.hist = ''  
            
        else:
            task_end_time = time_task_start + self.graph.tasks[task_to_add]['Data']
            
            self.parent = parent
            self.tasks_done_time = deepcopy(parent.tasks_done_time)
            self.tasks_done_time[task_to_add] = task_end_time

            self.cores = deepcopy(parent.cores)
            self.cores[core_where_to_add] = {"task" : task_to_add, "task_end_time" : task_end_time}
                
            self.g = self.compute_g()
            self.f = self.g + self.h()
            
            self.hist = parent.hist + f"|Task {task_to_add} start at time {time_task_start} on core {core_where_to_add} "
                 
    def __repr__(self):
        string = '[' + ','.join([n2letter(task) for task in self.tasks_done_time]) + ']'
        string += ''.join([f"({core['task']} end at {core['task_end_time']})" for core in self.cores.values()])
        return string
            
    def is_goal(self):
        '''Return whether a node is a full schedule'''
        return len(self.tasks_done_time) == task_count
    
    def successors(self):                     
        '''Create and return list of child node of self'''
        childs = list()
        
        #On regarde toutes les tâches qu'on va tenter de rajouter
        for task, info in self.graph.tasks.items():
            
            #On passe les taches déjà ajoutées
            if task in self.tasks_done_time: 
                continue
            
            #On ne garde que les taches dont toutes les dépendances ont été réalisées
            if not all([task_required in self.tasks_done_time for task_required in info['Dependencies']]): 
                continue
            
            #On calcul le temps ou toutes les dépendances de task seront terminés par les coeurs   
            time_all_tasks_done = max([0] + [self.tasks_done_time[task_required] for task_required in info['Dependencies']])
                                         
            for core_n, core in self.cores.items():
                #On ne commence à faire la task que lorsque toutes les dépendances sont calculées et que le core est disponible.
                time_core_available = core["task_end_time"]
                time_task_start = max(time_all_tasks_done, time_core_available)
                
                child = Node(parent = self, task_to_add=task, core_where_to_add=core_n, time_task_start=time_task_start)    
                childs.append(child)
                
        return sorted(childs, key = lambda node: node.f)
        
    def cost(self, child_node):
        '''Return the cost of going from self to child_node, a child node of self
        '''
        res = child_node.g - self.g
        if res < 0:
            raise Exception("Cost difference is negative")
        return res
    
    def h(self):
        '''Estimated remaining time of the node-schedule for reaching a terminal node. Must understimate true value.
        '''
        successor_tasks = list()
        for task, info in self.graph.tasks.items():
            if task in self.tasks_done_time: #On passe les taches déjà ajoutées
                continue
            if not all([task_required in self.tasks_done_time for task_required in info['Dependencies']]):   #On ne garde que les taches dont toutes les dépendances ont été réalisées
                continue
            successor_tasks.append(task)
        if len(successor_tasks) == 0:
            return 0
        return max([self.graph.tasks_to_sbl[task] for task in successor_tasks])
        
    
    #Node-schedule method
    def __lt__(self, node):
        return self.f < node.f
    
    def __hash__(self):
        return self.f
        
    def __eq__(self, node):
        '''Return whether a node is equal to another. Two nodes are considered equal if they have completed the same tasks and if all their cores stop working at same time.
        '''
        if self.g != node.g:
            return False            
        
        return self.set_of_core() == node.set_of_core() and self.tasks_done_time == node.tasks_done_time
        
    def set_of_core(self):
        return set([(core["task"], core["task_end_time"]) for core in self.cores.values()])
    
    def compute_g(self):
        return max([core["task_end_time"] for core in self.cores.values()])
    
# r = Node()
# x,y = r.successors()
# a,b,c,d,e,f, *_ = x.successors()[0].successors()
# g, h, i, j, k, l, *_ = y.successors()[0].successors()

# #Test successors
# print(a, b, c, d, e, f, sep = '\n')
# print()
# print(g, h, i, j, k, l, sep = '\n')
# print()

# #Test __eq__
# for i in (g, h, i, j, k, l):
#     for j in (a, b, c, d, e, f):
#         if i == j:
#             print(i, j)

# #Test cost
# x.cost(a)

# #Test h
# print(r.h())
# print(x.h())



In [819]:
import time
import queue
import random as rd

class A_star():
    def __init__(self, root, graph):
        self.root = root
        self.graph = graph

    def find_best_path(self, max_time = float('inf')):
        t0 = time.time()
        
        OPEN_QUEUE = queue.PriorityQueue()
        OPEN_QUEUE.put(self.root)       #Open queue, pile of most urgent node to be evaluated
        OPEN_DICT = {self.root: None}   #Open set, more efficient way to compute if a node is in the open list
        CLOSED_DICT = dict()            #Closed list, list of already explored node
        
        while not OPEN_QUEUE.empty():
            if time.time() - t0 > max_time:
                print('Time out')
                return

            current = OPEN_QUEUE.get()
            del OPEN_DICT[current]
            CLOSED_DICT[current] = None
            if current.is_goal():   #If we reach a final node, it is the optimal solution and we return it
                return current

            for child in current.successors():

                if child in CLOSED_DICT:    #We pass the node already visited
                    continue
                    
                if child in OPEN_DICT:      #We pass the node already waiting to be visited (note: in A* general, we need to recompute child.g and child.parent here, but for us child.g only depends on child (not on parent))
                    continue
                
                
                else:
                    # child.g = current.g + current.cost(child)     #General method. In our case, node.g does not depend on the path and the parent node
                    OPEN_QUEUE.put(child)
                    OPEN_DICT[child] = None
                    
        raise Exception("No path from root to a terminal node")
    
# A_star(root = Node(), graph=graph).find_best_path()

In [820]:
import cProfile
import pstats

root = Node()
a_star = A_star(root = root, graph=graph)

with cProfile.Profile() as pr:
    final_node = a_star.find_best_path(max_time=10)

print(final_node)
stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
stats.dump_stats(filename='profiling.prof')
stats.print_stats()

Time out
None
         26033602 function calls (24350690 primitive calls) in 10.001 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  3050590    1.832    0.000    2.937    0.000 <ipython-input-818-8258771efe32>:111(set_of_core)
  5085442    1.612    0.000    4.549    0.000 <ipython-input-818-8258771efe32>:103(__eq__)
1682912/88812    1.344    0.000    3.006    0.000 C:\Users\timot\AppData\Local\Programs\Python\Python39\lib\copy.py:128(deepcopy)
        1    1.017    1.017   10.001   10.001 <ipython-input-819-ddd3af7057e0>:10(find_best_path)
  3050590    0.868    0.000    0.868    0.000 <ipython-input-818-8258771efe32>:112(<listcomp>)
177624/88812    0.727    0.000    2.771    0.000 C:\Users\timot\AppData\Local\Programs\Python\Python39\lib\copy.py:226(_deepcopy_dict)
  3365824    0.296    0.000    0.296    0.000 {method 'get' of 'dict' objects}
    44406    0.291    0.000    0.443    0.000 <ipython-input-818-8258771efe32>:81

In [821]:
c = deepcopy(fn)
while c is not None:
    print(c)
    c = c.parent

[a,b,d,e,g,h,i,f,j,c](3 end at 22440)(6 end at 20276)
[a,b,d,e,g,h,i,f,j](10 end at 19699)(6 end at 20276)
[a,b,d,e,g,h,i,f](9 end at 16816)(6 end at 20276)
[a,b,d,e,g,h,i](9 end at 16816)(8 end at 15160)
[a,b,d,e,g,h](5 end at 11564)(8 end at 15160)
[a,b,d,e,g](5 end at 11564)(7 end at 10887)
[a,b,d,e](5 end at 11564)(4 end at 7009)
[a,b,d](2 end at 6499)(4 end at 7009)
[a,b](2 end at 6499)(-1 end at 0)
[a](1 end at 2843)(-1 end at 0)
[](-1 end at 0)(-1 end at 0)


In [822]:
2843+3656+5065 


11564

In [823]:
2843 + 4166+3878

10887

In [824]:
stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
stats.dump_stats(filename='profiling.prof')

In [825]:
# final_node.hist

In [826]:

2843 + 2883+3596+5065+3656 + 2741

20784

In [827]:
2843 + 3878 + 5252 + 4166 + 5116

21255

In [828]:
20784 - 21255


-471